In [234]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.font_manager

from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest

from sklearn.cluster import DBSCAN

from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import pydot

In [208]:
from pylab import rcParams 
rcParams['figure.figsize'] = 20,20 #увеличиваем размер графиков
pd.options.display.max_rows = 100 #увеличиваем количество отображаемых строк
pd.options.display.max_columns = 1000 #увеличиваем количество отображаемых колонок

**Задание к домашней работе**

1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. Использовать датасет www.kaggle.com...iques/data (train.csv)
2. Данных немного, поэтому необходимо использовать 10-fold кросс-валидацию для оценки качества моделей
3. Построить случайный лес, вывести важность признаков
4. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
5. Для валидации модели 2-го уровня использовать отдельный hold-out датасет, как на занятии
6. Показать, что использование ансамблей моделей действительно улучшает качество (стекинг vs другие модели сравнивать на hold-out)

In [256]:
houses = pd.read_csv('train.csv', delimiter = ',')

Для начала проводим изучение самого датасета.

In [263]:
houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

**Выводы:**

1. Датасет содержит как категориальные значения, так и массивы чисел.
2. Датасет содержит кучу NaN
3. Предсказываемая колонка SalePrice также не относится к категориальным
Т.о.:
    -очистим датасет от NaN 
    - разделим датасет на две части, чтобы выделить конкретные категориальные колонки от некатегориальных
    - переделаем SalePrice в категориальную, чтобы было проще предсказывать.

In [288]:

df = houses

non_categorical_cols = []

columns = df.columns
for col in columns:
    if df[col].dtype != 'object':
        non_categorical_cols.append(col)


df_non_cat = df[non_categorical_cols]
catigorical = df[[x for x in columns if x not in non_categorical_cols]]
non_cat_full = df_non_cat.fillna(df_non_cat.mean())

non_cat_full.head(5)
# catigorical.info()
cat = catigorical.drop(['Alley', 'FireplaceQu', 'PoolQC', 'MiscFeature', 'Fence'], 1) #
cat = pd.concat([cat, non_cat_full['SalePrice']], axis=1)
cat = cat.dropna(axis=0)
cat.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 0 to 1459
Data columns (total 39 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1338 non-null   object
 1   Street         1338 non-null   object
 2   LotShape       1338 non-null   object
 3   LandContour    1338 non-null   object
 4   Utilities      1338 non-null   object
 5   LotConfig      1338 non-null   object
 6   LandSlope      1338 non-null   object
 7   Neighborhood   1338 non-null   object
 8   Condition1     1338 non-null   object
 9   Condition2     1338 non-null   object
 10  BldgType       1338 non-null   object
 11  HouseStyle     1338 non-null   object
 12  RoofStyle      1338 non-null   object
 13  RoofMatl       1338 non-null   object
 14  Exterior1st    1338 non-null   object
 15  Exterior2nd    1338 non-null   object
 16  MasVnrType     1338 non-null   object
 17  ExterQual      1338 non-null   object
 18  ExterCond      1338 non-null

In [289]:
Y_h = cat['SalePrice'] # Y с набором числовых данных
X_cat = cat.drop(['SalePrice'], 1) # Х с набором ТОЛЬКО КАТЕГОРИАЛЬНЫХ данных
Y_h.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: SalePrice, dtype: int64

In [290]:
Y_cat = 0
Y_cat = np.where((Y_h > 35311.000000)&(Y_h <= 135000.000000), 1, Y_cat)
Y_cat = np.where((Y_h > 135000.000000 )&(Y_h <=168500.000000), 2, Y_cat)
Y_cat = np.where((Y_h > 168500.000000 )&(Y_h<= 220000.000000), 3, Y_cat)
Y_cat = np.where((Y_h > 220000.000000 ), 4, Y_cat)
Y_cat

array([3, 3, 4, ..., 4, 2, 2])

In [291]:
Y_cat = pd.DataFrame(Y_cat, columns=['Prices Categories'])

Y_cat.head()# Y с набором ТОЛЬКО КАТЕГОРИАЛЬНЫХ данных

,Prices Categories
0,3
1,3
2,4
3,2
4,4


Применим к колонкам one hot

In [269]:
def get_one_hot(X, cols):
    for each in cols:
        dummies = pd.get_dummies(X[each], prefix=each, drop_first=True)
        X = pd.concat([X, dummies], axis=1)
    return X

In [275]:
def get_score(X,y, random_seed=42, model=None):
    if model is None:
        model = LogisticRegression()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed )  
        model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [292]:
X_hot = get_one_hot(X_cat, X_cat)
X_hot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338 entries, 0 to 1459
Columns: 229 entries, MSZoning to SaleCondition_Partial
dtypes: object(38), uint8(191)
memory usage: 657.2+ KB


Удалим исходные (по идее, параметр drop_first=True в функции get_one_hot должен это делать автоматом, но, почему-то этого е происходит)

In [293]:
X_hot = X_hot.drop(['MSZoning', 'Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive','SaleType','SaleCondition'], 1)

X_hot.head()

,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_TA,Foundation_CBlock,Foundation_PConc,Foundation_Stone,Foundation_Wood,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasW,Heating_Grav,Heating_OthW,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
2,0,0,1,0,1,0,0,0,0,0,1,0,0,0,

Чисто для себя проверим качество модели, применив логистическую регрессию:

In [294]:
get_score(X_hot,Y_cat)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6393034825870647

**10-fold model**

In [300]:
from numpy import mean
from numpy import std
# from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

# готовим перекрестную проверку
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# Обучаем модель
model = LogisticRegression()
# Оцениваем модель
scores = cross_val_score(model, X_hot, Y_cat, scoring='accuracy', cv=cv, n_jobs=-1)
# выводим качество
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.654 (0.019)


In [220]:
X_train, X_test, y_train, y_test = train_test_split(X_hot, Y_cat, test_size=0.3, random_state=5 )

In [221]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve

**Random forest** 

In [346]:
from sklearn.ensemble import RandomForestClassifier
RSEED = 55
# создаем лес с 100 деревьями
model = RandomForestClassifier(n_estimators=100, 
                               random_state=RSEED, 
                               max_features = 'sqrt',
                               n_jobs=-1, verbose = 1)

In [347]:
model.fit(X_train, y_train)

<ipython-input-347-d768f88d541e>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished


RandomForestClassifier(max_features='sqrt', n_jobs=-1, random_state=55,
                       verbose=1)

In [348]:
model.score(X_train, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


0.9957264957264957

In [355]:
model.score(X_test, y_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


0.6268656716417911

Модель явно перучивается на тренировочных данных

**Важность признаков**

In [350]:
features = list(X_train.columns)
# Извлекаем значимость параметров
fi = pd.DataFrame({'Признак': list(X_train.columns),
                   'Важность': model.feature_importances_}).\
                    sort_values('Важность', ascending = False)

# Выводим значения
fi.head(10)


,Признак,Важность
113,ExterQual_TA,0.034286
167,GarageFinish_Unf,0.032184
123,BsmtQual_TA,0.027062
154,KitchenQual_TA,0.026135
122,BsmtQual_Gd,0.025914
153,KitchenQual_Gd,0.025726
7,LotShape_Reg,0.023101
109,MasVnrType_None,0.021119
65,HouseStyle_2Story,0.021006
131,BsmtFinType1_GLQ,0.019713


Попробуем визуализировать лес:

In [351]:
from sklearn.tree import export_graphviz

estimator = best_model.estimators_[1]

# Экспортируем дерево из леса
export_graphviz(estimator, 'forest.dot', rounded = True, 
                feature_names=X_train.columns, max_depth = 8, 
                class_names = str(estimator.classes_), filled = True)

In [353]:
from subprocess import call
from IPython.display import Image
call(['dot', '-Tpng', 'forest.dot', '-o', 'forest.png', '-Gdpi=400'])

FileNotFoundError: [WinError 2] Не удается найти указанный файл

Вот тут какая-то непонятная проблема вылезает, у меня почему-то он не видит .dot файлю Я устал с этим всем бороться, если честно.


**Уважаемые кураторы, проверяющие ДЗ!**

Дальше по ДЗ я ничего не сделал, поскольку:

    - как было замечено в  комментариях к ДЗ от других участников, в самом уроке нет ни слова ни об одном из пунктов ДЗ. Я конечно понимаю, что очень важно и необходимо самостоятельно изучать часть информации, но илчно я в процессе столкнулся с массой сложностей, прямо вообще не связанных с ДЗ (ну, вот та ошибка, которую вы видите выше - яркий тому пример), а ответов я не увидел ни в slack, ни где бы то ни ыло еще
    - даже если делать дальше, то я вообще не понимаю, о чем речь. Вы бы хотя бы доп фал какой дали для самостоятельного изучения. Если вы думаете, что у большинства ваших учеников есть возможность только заниматься изучением, то вы ошибаетесь. поверьте, я был бы очень рад этому, но вот лично у меня, то командировки (и так уже на 4 месяца отстал), то еще что-то. Как-то еще и пожить охота, если что.

В общем, прошу дать РАЗВЕРНУТЫЕ КОММЕНТАРИИ по каждому пункту ДЗ.
Ребят, я сам являюсь куратором в одной из онлайн-школ (пусть и не в IT, а в финансах и инвестициях). И несмотря на то, что прекрасно осознаю важность самостоятельного изучения части материалов, а также то, что подавляющее большинство ваших заданий действительно решаются достаточно просто, уверен что конкретно вот такие ДЗ, как это без предварительной информационной подготовки просто недопустимы. Это, конечно, если вы хотите иметь высокую эффективность усвоения знаний своих учеников без выгорания. Думаю, что это интересно и потенциальным работодателям. Ни в коем случае не претендую на мнение последней инстанции и считаю, что вы все равно лучшие.

<p style="align: center;"><img align=center src="https://c.tenor.com/xuv_LKjUjLsAAAAd/robert-downey-jr-roses.gif"  width=500></p>